In [ ]:
import sys
import os
import numpy as np
sys.path.append(os.path.abspath("../src"))
from bandit_gpu import load_cost_curves, run_bandit_gpu


In [ ]:
# Load damage and protection cost curves for the chosen city

damage_file = "../data/Damage_cost_curves.tab"
protection_file = "../data/Protection_cost_curves_high_estimate.tab"  # choose high estimate curves
city = "Halmstad"
years_range = (2025, 2100)

heights, damage_costs, protection_costs = load_cost_curves(
    damage_file, protection_file, city
)
print(f"Loaded {len(heights)} height levels from 0 to 12 m")
print("Candidate heights (m):", heights)

Loaded 25 height levels from 0 to 12 m
Candidate heights (m): [ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12. ]


In [ ]:
# Load predictive posterior and latent MSL paths from the mixture NPZ

pp = np.load("../data/pp_inputs_halmsdad_pp_mixture_2025_2100.npz")
posterior_params = {
    "eta0": pp["eta0"],
    "eta1": pp["eta1"],
    "alpha0": pp["alpha0"],
    "xi": pp["xi"],
    "u": float(pp["u"]),
}
years_future = pp["years_future"]
X_future_paths = pp["X_future_paths"]

print(f"Loaded {len(posterior_params['eta0'])} posterior draws and {X_future_paths.shape[0]} MSL paths")

Loaded 64000 posterior draws and 20000 MSL paths


In [ ]:
# Run the GPU bandit without deterministic pruning

candidate_indices = list(range(len(heights)))
print("Number of candidate heights:", len(candidate_indices))

best_height, history = run_bandit_gpu(
    heights=heights,
    damage_costs=damage_costs,
    protection_costs=protection_costs,
    candidate_indices=candidate_indices,
    years_all=years_future,
    X_pred_paths_cm=X_future_paths,
    posterior_params=posterior_params,
    years_range=years_range,
    delta=0.05,
    max_rounds=1000000000,      # total number of simulated flood scenarios
    verbose=True,
    chunk_size=90000,       # GPU batch size; adjust based on available memory
    check_every=1000000,      # how often to recompute confidence intervals
)

print(f"Selected design height: {best_height:.2f} m")
print("History (last few entries):")
print(history['rounds'][-3:])
print(history['best_height'][-3:])


Number of candidate heights: 25

Round 90000 – all candidate options:
  option  0: idx=0, height=0.00 m, mean_total=2047.606, CI=[-1531.116, 5626.329]
  option  1: idx=1, height=0.50 m, mean_total=2066.134, CI=[-1512.588, 5644.857]
  option  2: idx=2, height=1.00 m, mean_total=2034.022, CI=[-1544.700, 5612.745]
  option  3: idx=3, height=1.50 m, mean_total=1463.624, CI=[-2115.098, 5042.347]
  option  4: idx=4, height=2.00 m, mean_total=929.315, CI=[-2649.407, 4508.038]
  option  5: idx=5, height=2.50 m, mean_total=720.456, CI=[-2858.267, 4299.179]
  option  6: idx=6, height=3.00 m, mean_total=684.676, CI=[-2894.046, 4263.399]
  option  7: idx=7, height=3.50 m, mean_total=727.091, CI=[-2851.631, 4305.814]
  option  8: idx=8, height=4.00 m, mean_total=823.048, CI=[-2755.675, 4401.770]
  option  9: idx=9, height=4.50 m, mean_total=951.621, CI=[-2627.102, 4530.343]
  option 10: idx=10, height=5.00 m, mean_total=1107.882, CI=[-2470.840, 4686.605]
  option 11: idx=11, height=5.50 m, mean_tot